In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import gc
import re
import csv
import os
import gc
import torch
# set to "cuda:1" for running in parallel on both GPUs
device = torch.device("cuda:1")
torch.cuda.set_device(device)
torch.set_default_device(device)
import Stemmer
import pandas as pd
from tqdm import tqdm
from utils.graph import KGraphPreproc
from utils.graph.chain import GraphChain
from utils.llm.mistral import MistralLLM
from utils.file import export_results_to_file
from utils.prompt import GRAPH_QA_PROMPT, ENTITY_PROMPT

In [9]:
gc.collect()
torch.cuda.empty_cache()

In [4]:
def get_response(prompt):
    # global chain
    # del mistral
    gc.collect()
    torch.cuda.empty_cache()
    r = chain.invoke(prompt)
    return r["result"]


def save_results(fpath, data_rows):
    with open(fpath, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["Model"])
        for r in data_rows:
            writer.writerow([str(r)])

In [5]:
mistral = MistralLLM()

In [6]:
metaqa_graph = KGraphPreproc.get_metaqa_graph()

chain = GraphChain.from_llm(
    llm=mistral,
    graph=metaqa_graph,
    qa_prompt=GRAPH_QA_PROMPT,
    entity_prompt=ENTITY_PROMPT,
    verbose=False,
)
chain.sbert_cache_path = "/datasets/MetaQA/cache/sbert.csv"


In [7]:
for hop in ["1hop", "2hop", "3hop"]:
    print(hop)
    # load q's
    metaqa = pd.read_csv(f"/datasets/MetaQA/{hop}/test_1000.txt", header=None, index_col=0)
    metaqa.rename(columns={1: "Question", 2: "Answers"}, inplace=True)


    for depth in [4]:
        print(f"depth: {depth}")
        # set the depth
        chain.exploration_depth = depth
        # init experiment
        experiment_name = f"sbert-kb{depth}"
        res_path = f"/datasets/MetaQA/results/{hop}/{experiment_name}.csv"
        results = []
        id_list = []
        l = 0
        # load if preinit'ed
        if os.path.isfile(res_path):
            r_df = pd.read_csv(res_path)
            l = len(r_df)
            results = list(r_df.Model.values)
        # run through
        for c, (i, r) in enumerate(tqdm(list(metaqa.iterrows()))):
            id_list.append(i)
            if c < l:
                continue
            q = r.Question
            response = get_response(q)
            results.append(response)
            # backup every 10 qs
            if c % 10 == 0:
                export_results_to_file(res_path, results, id_list)
        export_results_to_file(res_path, results, id_list)

1hop
depth: 4


100%|██████████| 1000/1000 [00:00<00:00, 1698098.79it/s]


2hop
depth: 4


100%|██████████| 1000/1000 [00:00<00:00, 1813361.00it/s]


3hop
depth: 4


  0%|          | 0/1000 [00:00<?, ?it/s]/model/utils/graph/chain.py:111: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  entity_string = self.entity_extraction_chain.run(question)


Checking embedding cache
Loading embedding cache


124680it [04:32, 457.96it/s]
/model/utils/graph/chain.py:148: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain(
100%|██████████| 1000/1000 [4:05:31<00:00, 14.73s/it] 
